# Diffe-Hellman

**Problema:** hay poder comunicarnos de forma simetrica para luego comunicarnos de forma asimetrica. Para ello se utiliza un protocolo de intercambio de llaves. 

## Protocolo de intercambio de llaves

Sea $(\mathbf{G}, \dot)$ un grupo cícilico de orden primo $q$ generado por $g \in \mathbf{G}$. El elemento generador es un elemento $g$ de un grupo ciclico $(G, \dot)$ tal que haciendo potencias $g^0, g^1, ...$ genera todos los elementos del grupo.

![diffie-hellman-key-exchange](../img/diffie-hellman-key-exchange.png)

Tanto Alice como Bob tienen $w = g^{\alpha \beta}$ como secreto compartido. 

## Problema del logaritmo discreto

El atacante tiene 
- $g$ que es publico.
- $u = g^\alpha$ y $v = g^\beta$ que estan en el canal.
- El secreto compartido es $w = g^{\alpha \beta}$
- Si hacemos logarimo base $g$, podemos obtener alpha y beta. 
    $$
    log_g(g^\beta) = \beta
    $$

    $$
    log_g(u) = \alpha
    $$

    Sin embargo, esto no es tan fácil, pues estamos trabajando con números discretos en un grupo ciclo.

In [59]:
import math
from typing import Tuple


In [128]:
def egcd(x: int, y: int) -> Tuple[int, int, int]:
    """Algoritmo de euclides extendido

    Args:
        x (int)
        y (int)

    Returns:
        (r: int, s:int, t:int) tal que s*x + t*y = r = mdc(x, y)
    """
    r2, r1 = (y, x) if x < y else (x, y)

    s1, s2 = 1, 0
    t1, t2 = 0, 1

    while r2 != 0: 
        q = r1 // r2
        r = r1 % r2 

        r1, s1, t1, r2, s2, t2 = r2, s2, t2, r, s1 - s2*q, t1 - t2*q

    return r1, s1, t1

def modinv(a, b):

    # El pequeño teorema de ferman 
    # Si mcd(a, p) = 1
    # a ^ p - 1 = 1 mod p     
    r, s, _ = egcd(a, b)

    if r != 1:
        raise Exception("Inverse doesn't exist")
    
    return (s % b + b) % b


In [129]:
class DiscreteLog:
    
    @staticmethod
    def bruteForce(g: int, y: int, n: int) -> int:
        """Búsqueda Exaustiva.

        Args:
            g (int): generador
            y (int): y = g^x
            n (int): orden del grupo G

        Returns:
            int: log_g(y)
        """
        x = 0
        while pow(g, x, n) != y: 
            x += 1
        return x
    

    @staticmethod
    def babyStep_GigantStep(
            g: int, 
            y: int, 
            n: int, 
            debug_log = False
        ) -> int:
        
        """Búsqueda Exaustiva "más inteligente" introducciendo una tabla de memoria. Se pasa en que dando y = g**x, se puede escribri x = i_x * m + j_x donde 0 <= i_x, j_x. Entonces y = g**{i_x*m}g**{j_x} con m >= raiz(n)

        Args:
            g (int): generador
            y (int): y = g^x
            n (int): orden del grupo G

        Returns:
            int: log_g(y)
        """
        m = math.ceil(math.sqrt(n))
        if debug_log: print("m = ", m)
        
        # T Es una tabla hash para almacenar tuplas (a, b) \in G X Z
        # a es usada como clave indice
        T = {}

        # Computar (g**j, j) para todo 0 <= j <= m
        for j in range(m):
            T[pow(g, j, n)] =  j  

        b = pow(modinv(g, n), m, n)
        
        _y = y 
        x = None

        # Computar y * (g**{-m})**i para i = 0, 1, 2, ... hasta entonces i_x con y * (g**{-m})**i_x
        for i in range(m):
            if _y in T.keys(): 
                x = (i * m + T[_y]) % n
                if debug_log: print(f"ix = {i} | jx = {T[_y]}")
                return x
            
            _y = (_y * b) % n
        return x 


    @staticmethod
    def pollardsRho(g: int, y: int, n: int) -> int: 
        pass

In [138]:
assert(DiscreteLog.bruteForce(3, 57, 113) == 100), "Caso de pre-prueba 1"
assert(DiscreteLog.bruteForce(13, 5732, 38629) == 23718), "Caso de pre-prueba 2"

In [135]:
assert(DiscreteLog.babyStep_GigantStep(3, 57, 113) == 100), "Caso de pre-prueba 1"
assert(DiscreteLog.babyStep_GigantStep(13, 5732, 38629) == 23718), "Caso de pre-prueba 2"

assert(DiscreteLog.babyStep_GigantStep(19, 1249446553, 3129553951) == 972), "Caso de prueba 1"
assert(DiscreteLog.babyStep_GigantStep(19, 1998919620, 3129553951) == 666261645), "Caso de prueba 2"
assert(DiscreteLog.babyStep_GigantStep(19, 1132858435, 3129553951) == 320275002), "Caso de prueba 3"
assert(DiscreteLog.babyStep_GigantStep(19, 2346346458, 3129553951) == 1322464706), "Caso de prueba 4"
assert(DiscreteLog.babyStep_GigantStep(19, 743603545, 3129553951) == 1644500154), "Caso de prueba 5"